## Data Loading

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlalchemy as sqla
import csv
import matplotlib.pyplot as plt

DB_USER = "postgres"
DB_PASSWORD = "a"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "postgres"

connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Créer un moteur SQLAlchemy
engine = sqla.create_engine(connection_string)

try:
    with engine.connect() as connection:
        print("Connected to PostgreSQL successfully!")
except Exception as e:
    print(f"Error: {e}")

connection = engine.connect()

query = """
SELECT 
    td.id,
    td.departure_date,
    td.train_number,
    td.relation,
    td.train_service,
    td.ptcar_number,
    td.line_number_departure,
    td.real_time_arrival,
    td.real_time_departure,
    td.planned_time_arrival,
    td.planned_time_departure,
    td.delay_arrival,
    td.delay_departure,
    ptcar.id AS ptcar_id,     
    ptcar.name AS name_travel,    
    td.line_number_arrival,
    dep.name AS departure_station_name,
    arr.name AS arrival_station_name,
    ptcar.longitude AS ptcar_longitude,  
    ptcar.latitude AS ptcar_latitude  
FROM train_data td
JOIN STATIONS ptcar ON td.ptcar_name = ptcar.id
JOIN STATIONS dep ON td.station_departure = dep.id
JOIN STATIONS arr ON td.station_arrival = arr.id
WHERE td.station_arrival = '487'  
  AND td.station_departure = '159'  
ORDER BY td.id;
"""

line = pd.read_sql(sqla.text(query), connection)
line =line.drop(columns=['train_service', 'relation'])

road = [159,151,149,150,156,157,154,245,773,132,310,365,422,300,628,614,591,
 540,118,178,235,281,480,108,642,625,623,557,558,384,199,559,645,123,195,73,
 261,560,187,438,172,342,341,80,503,629,267,313,608,339,452,564,493,
 317,704,757,70]

print("Road:", len(road))

tempo = []
clean = []
road_index = 0

for i, row in line.iterrows():
    ptcar_id = row['ptcar_id']
    if ptcar_id == road[road_index]:
        tempo.append(row)
        road_index += 1
        if len(tempo) == len(road):
            clean.extend(tempo)
            tempo = []
            road_index = 0
    else:
        tempo = []
        road_index = 0

clean_df = pd.DataFrame(clean)

query = """
    SELECT * FROM TYPE_DAY;
"""

type_day = pd.read_sql(sqla.text(query), connection)
clean_df= clean_df.drop(columns=['departure_station_name','arrival_station_name','id', 'name_travel', 'ptcar_number','ptcar_id'])
merged_df = pd.merge(clean_df, type_day, left_on='departure_date', right_on='date', how='left')
merged_df.loc[merged_df.index[::57], "delay_arrival"] = 0

merged_df.loc[merged_df.index[::57], "real_time_arrival"] = merged_df.loc[merged_df.index[::57], "real_time_departure"]
merged_df.loc[merged_df.index[::57], "planned_time_arrival"] = merged_df.loc[merged_df.index[::57], "planned_time_departure"]
merged_df.loc[merged_df.index[::57], "line_number_arrival"] = merged_df.loc[merged_df.index[::57], "line_number_departure"]


merged_df.loc[merged_df.index[56::57], "line_number_departure"] = 162
merged_df.loc[merged_df.index[56::57], "line_number_arrival"] = 162

index = merged_df.index[56::57]
mask = merged_df.loc[index, "real_time_departure"] == None
merged_df.loc[index, "real_time_departure"] = merged_df.loc[index, "real_time_arrival"]

mask = merged_df.loc[index, "planned_time_departure"] == None
merged_df.loc[index, "planned_time_departure"] = merged_df.loc[index, "planned_time_arrival"]

mask = merged_df.loc[index, "delay_departure"] == None
merged_df.loc[index, "delay_departure"] = merged_df.loc[index, "delay_arrival"]
#merged_df = merged_df.ffill(axis=1).bfill(axis=1)

query = """
SELECT 
w.date,
EXTRACT(HOUR FROM w.hour) AS hour_of_day,
AVG(w.temperature) AS avg_temperature,
AVG(w.dewpoint) AS avg_dewpoint,
AVG(w.relative_humidity) AS avg_relative_humidity,
AVG(w.precipitation) AS avg_precipitation,
AVG(w.snowfall) AS avg_snowfall,
AVG(w.wind_direction) AS avg_wind_direction,
AVG(w.wind_speed) AS avg_wind_speed,
AVG(w.pressure) AS avg_pressure
FROM WEATHER w
GROUP BY w.date, hour_of_day
ORDER BY w.date, hour_of_day;
"""

weather = pd.read_sql(sqla.text(query), connection)

print("In Loading...")

merged_df['time'] = pd.to_datetime(merged_df['planned_time_arrival'], format='%H:%M:%S')
merged_df['hour'] = merged_df['time'].dt.hour
final_df = pd.merge(merged_df, weather, left_on=['departure_date', 'hour'], right_on=['date', 'hour_of_day'], how='left')
final_df = final_df.drop(columns=['date_x', 'date_y', 'hour_of_day','time','hour'])

def parse_time_to_seconds(val):
    if pd.isna(val):
        return 0
    try:
        if isinstance(val, (int, float)):
            return int(val)
        if str(val).isdigit():
            return int(val)
        t = pd.to_datetime(val, format='%H:%M:%S', errors='coerce')
        if pd.isna(t):
            return 0
        return t.hour * 3600 + t.minute * 60 + t.second
    except Exception:
        return 0


final_df['departure_date'] = pd.to_datetime(final_df['departure_date'], format='%Y-%m-%d')

final_df['departure_year'] = final_df['departure_date'].dt.year
final_df['departure_month'] = final_df['departure_date'].dt.month
final_df['departure_day'] = final_df['departure_date'].dt.day
final_df['departure_weekday'] = final_df['departure_date'].dt.weekday

final_df['real_time_arrival'] = final_df['real_time_arrival'].apply(parse_time_to_seconds)
final_df['planned_time_arrival'] = final_df['planned_time_arrival'].apply(parse_time_to_seconds)
final_df['planned_time_departure'] = final_df['planned_time_departure'].apply(parse_time_to_seconds)
final_df['real_time_departure'] = final_df['real_time_departure'].apply(parse_time_to_seconds)

final_df = final_df.drop(columns=['departure_date', 'line_number_departure', 'line_number_arrival' ,'train_number'])
final_df['delay_arrival'] = final_df['delay_arrival'].apply(parse_time_to_seconds)

trajet_valid = []

for i in range(0, len(final_df), 57):
    trajet = final_df.iloc[i:i+57]
    if trajet.shape[0] < 57:
        print("Not enough data for this trajet, skipping")
        continue

    if trajet.isnull().values.any():
        print(f"Skipping trajet starting at index {i}: contains NaN")
        continue

    if trajet['delay_arrival'].abs().max() > 36000:
        print("Delay arrival exceeds 10000 seconds, skipping")
        continue

    trajet_valid.append(trajet)

trajet_valid_df = pd.concat(trajet_valid, ignore_index=True)
trajet_valid_df["stop_index"] = trajet_valid_df.index % 57

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

stop = 57
features = 23
travels = trajet_valid_df.shape[0] // stop
print("Travels:", travels)
print("Features:", features)
print("Stop:", stop)


In [ ]:
data = final_df.values.reshape(travels,stop,features)
print(data.shape)
indices = np.arange(data.shape[0])

In [ ]:
train_size = int(0.8 * len(indices))
test_size = len(indices) - train_size

train_data = data[:train_size]
test_data = data[train_size:]

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)

## Dataset

In [ ]:
class Travels(Dataset):
    def __init__(self,trips,input_length, target, target_index = 4):
        self.trips = trips
        self.input_length = input_length
        self.target = target
        self.target_index = target_index

        all_idx = list(range(self.trips.shape[2]))
        exclude_idx = [target_index]
        self.continuous_features_idx = [i for i in all_idx if i not in exclude_idx]
    def __len__(self):
        return self.trips.shape[0]
    
    def __getitem__(self, idx):
        trip = self.trips[idx]
        X_cont = trip[:self.input_length, :].astype(np.float32)
        
        y = trip[self.target, self.target_index]

        return (
            torch.tensor(X_cont, dtype=torch.float32),                     
            torch.tensor(y, dtype=torch.float32)
        ) 

class ModelRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(ModelRNN, self).__init__()

        self.input_size = input_size

        self.rnn = nn.RNN(input_size=self.input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x_cont):

        out,h_n  = self.rnn(x_cont)
        h_last = h_n[-1]
        return self.fc(h_last)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

input_length = 1
target = 1

trained = Travels(train_data, input_length, target)
test = Travels(test_data, input_length, target)

model = ModelRNN(input_size= features, hidden_size=  features*2, num_layers=2, output_size=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = model.to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 3

train_loader = DataLoader(trained, batch_size=4, shuffle=True)
test_loader = DataLoader(test, batch_size=4, shuffle=False)

In [ ]:
def train(num_epochs,model):
    print("Training...", num_epochs, "epochs", "model:", model)
    train_avg_loss = []
    test_avg_loss = []
    for epoch in range(num_epochs):
        train_loss = []
        model.train()
        for X_cont, y in train_loader:
            #X_cont = X_cont.to(device)
            #y = y.to(device)
            optimizer.zero_grad()
            outputs = model(X_cont)

            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
        train_avg_loss.append(sum(train_loss) / len(train_loss))
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_avg_loss[-1]:.4f}")
        model.eval()
        test_losses = []
        with torch.no_grad():
            for X_cont, y in test_loader:
                #X_cont = X_cont.to(device)
                #y = y.to(device)
                outputs = model(X_cont)
                loss = criterion(outputs, y)
                test_losses.append(loss.item())
        test_avg_loss.append(sum(test_losses) / len(test_losses))
        print(f"Epoch [{epoch+1}/{num_epochs}], Test Loss: {test_avg_loss[-1]:.4f}")

    return train_avg_loss, test_avg_loss

In [ ]:
train_avg_loss, test_avg_loss = train(num_epochs,model)